In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.contingency_tables as ct
import datetime

In [ ]:
benchmark_data_path = "../resources/BenchmarkDataset"
benchmark_dataset = os.listdir(benchmark_data_path)
# ---> final micro results with HTML formats
benchmark_save_path2 = "../resources/BenchmarkResults/BenchmarkResults_21_9_2023_new_save_format_HTMLFormat_redo/"
# ---> final micro results with 6 format
benchmark_save_path1 = "../resources/BenchmarkResults/BenchmarkResults_21_9_2023_new_save_format_redo/"
benchmark_dataset = [data for data in benchmark_dataset if data.endswith(
    ".csv") and "breast" not in data]
dfs = []
for bd in benchmark_dataset:

    pass_metrics = []
    print(bd)
    df1 = pd.read_csv(os.path.join(benchmark_save_path1, bd,
                      "Micro_"+bd+"_output_revamped.csv"))
    try:
        df2 = pd.read_csv(os.path.join(benchmark_save_path2,
                          bd, "Micro_"+bd+"_output_revamped.csv"))
        df = pd.concat([df1, df2], ignore_index=True)
    except:
        df = df1
    print(df1.shape, df2.shape, df.shape)
    df["Dataset-name"] = bd
    dfs.append(df)
all_data = pd.concat(dfs, ignore_index=True)
pivot_table = all_data.pivot_table(
    index=["TestCase", 'TableManipulation', "temperature"],
    columns='tableFormat',
    values=['pass_1', 'pass_3', 'pass_5', 'pass_10', 'pass_15', 'Result'],
    # Custom aggregation function to append values to a list
    aggfunc=lambda x: list(x)
)
attributes = {c: list(df[c].unique()) for c in df.columns if c in [
    "temperature", "tableFormat", "TableManipulation", "TestCase"]}

# RQ1

### Average pass @ 1, temp =0 Tests Vs formats across all datasets

In [ ]:
all_data_test_vs_format_on_original_data = all_data[(
    all_data["TableManipulation"] == "OriginalData") & (all_data["temperature"] == 0.0)]
test_vs_format_on_original_data = all_data_test_vs_format_on_original_data.pivot_table(
    index=["TestCase"],
    columns='tableFormat',
    values=['pass_1'],
    aggfunc=["mean"]  # Custom aggregation function to append values to a list
)
test_vs_format_on_original_data = np.round(
    test_vs_format_on_original_data, decimals=2)

In [ ]:
test_vs_format_on_original_data.T.to_clipboard()

## p-value RQ1

In [ ]:
import scipy
data_id = all_data[all_data["TableManipulation"].isin(["OriginalData"]) & (
    all_data["temperature"] == 0.0)] 
pivot_table_across_formats = data_id.pivot_table(
    index=["TestCase"],
    columns='tableFormat',
    values=['pass_1'],
    # Custom aggregation function to append values to a list
    aggfunc=lambda x: list(x)
)
p_val = {}
for id in range(pivot_table_across_formats.shape[0]):
    print(id)
    print(f"For Test: {pivot_table_across_formats.index[id]} pass@1 temp =0.1")
    sorted_list = [i[-1] for i in test_vs_format_on_original_data.T.sort_values(
        by=[pivot_table_across_formats.index[id]], ascending=False).index]
    sub_vals = {}
    index_val = pivot_table_across_formats.index[id]
    for x in range(1, len(sorted_list)):
        vals1 = pivot_table_across_formats.loc[index_val,
                                               ("pass_1", sorted_list[0])]
        vals2 = pivot_table_across_formats.loc[index_val,
                                               ("pass_1", sorted_list[x])]
        min_val = min(len(vals1), len(vals2))
        print(min_val)
        p_value = scipy.stats.ttest_rel(vals1[:min_val], vals2[:min_val])
        sub_vals[f"{sorted_list[0]}-{sorted_list[x]}"] = {"p-value": p_value.pvalue,
                                                          "statistics": p_value.statistic, "df": p_value.df, "tests_count": min_val}
    p_val[pivot_table_across_formats.index[id]] = sub_vals

In [ ]:
import json


class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, pd.Series):
            return obj.tolist()
        elif isinstance(obj, datetime):
            # Handle datetime objects
            return obj.isoformat()
        elif isinstance(obj, set):
            # Handle sets
            return list(obj)
        # Add more custom conversions for other data types if needed
        return super().default(obj)


with open("../resources/all_p_vals/p_vals_micro_tests_RQ1.json", "w") as f:
    json.dump(p_val, f, indent=3, cls=CustomJSONEncoder)

### Average pass@1, temp 0.0 over different data averaged across tests for different formats

In [ ]:
all_data_dataset_vs_format_on_original_data = all_data[(
    all_data["TableManipulation"] == "OriginalData") & (all_data["temperature"] == 0.0)]
dataset_vs_format_on_original_data = all_data_dataset_vs_format_on_original_data.pivot_table(
    index=["Dataset-name"],
    columns='tableFormat',
    values=['pass_1'],
    aggfunc=["mean"]  # Custom aggregation function to append values to a list
)
test_vs_format_on_original_data = np.round(
    dataset_vs_format_on_original_data, decimals=2)
test_vs_format_on_original_data

# RQ2

## All noise vs test across formats

In [ ]:
all_data["tableFormat"].value_counts().index

In [ ]:
all_data_test_vs_format_on_all_noise_ops = all_data[(
    all_data["temperature"] == 0.0)]
test_vs_format_on_all_noise_ops = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["tableFormat", "TableManipulation"],
    columns='TestCase',
    values=['pass_1'],
    aggfunc=["mean"]  # Custom aggregation function to append values to a list
)
test_vs_format_on_all_noise_ops_list = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["tableFormat", "TableManipulation"],
    columns='TestCase',
    values=['pass_1'],
    # Custom aggregation function to append values to a list
    aggfunc=lambda x: list(x)
)
test_vs_format_on_all_noise_ops = np.round(
    test_vs_format_on_all_noise_ops, decimals=2)

In [ ]:
noises = [
    'OriginalData',
    'ShuffleRows',
    'ShuffleColumns',
    'ShuffleColumnNames',
    'SequentialColumnNames',
    'ArbitraryColumnNames',
    'TransposeTable',
    'ColumnCluster',
    'SerializeTable']
table_formats = all_data["tableFormat"].value_counts().index
indi = [(formatType, noise)
        for formatType in table_formats for noise in noises]
cols = [(metric, "pass_1", test)
        for test in attributes["TestCase"] for metric in ["mean"]]
cols_agg = [("pass_1", test) for test in attributes["TestCase"]]
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops[cols]
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops.reindex(indi)
test_vs_format_on_all_noise_ops_list = test_vs_format_on_all_noise_ops_list[cols_agg]
test_vs_format_on_all_noise_ops_list = test_vs_format_on_all_noise_ops_list.reindex(
    indi)

In [ ]:
test_vs_format_on_all_noise_ops.head(10)

## RQ2: Delta values from original with p-values

In [ ]:
import scipy
test_vs_format_on_all_noise_ops_diff_with_pval = test_vs_format_on_all_noise_ops.copy(
    deep=True)
test_vs_format_on_all_noise_ops_pval = test_vs_format_on_all_noise_ops.copy(
    deep=True)
for i in test_vs_format_on_all_noise_ops_diff_with_pval.index:
    for j in test_vs_format_on_all_noise_ops_diff_with_pval.columns:
        ix = (i[0], "OriginalData")
        jj = (j[1], j[2])
        original_values = test_vs_format_on_all_noise_ops_list.loc[ix, jj]
        noise_induced_values = test_vs_format_on_all_noise_ops_list.loc[i, jj]

        if i[1] != "OriginalData":
            min_val = min(len(original_values), len(noise_induced_values))
            p_value = scipy.stats.ttest_rel(
                original_values[:min_val], noise_induced_values[:min_val])
            benoffi_corrected_p_val_threshold = 0.01/8
            subtract_from = test_vs_format_on_all_noise_ops.loc[ix, j]
            difference = test_vs_format_on_all_noise_ops_diff_with_pval.loc[i,
                                                                            j]-subtract_from
            if "-" in str(difference):
                str_diff = "{:.2f}".format(difference)
            else:
                str_diff = "+"+"{:.2f}".format(difference)
            if p_value.pvalue < benoffi_corrected_p_val_threshold:
                str_diff += "**"
            test_vs_format_on_all_noise_ops_diff_with_pval.loc[i, j] = str_diff
            test_vs_format_on_all_noise_ops_pval.loc[i, j] = p_value.pvalue

In [ ]:
test_vs_format_on_all_noise_ops_diff_with_pval.head(30)

#### only_diff

In [ ]:
test_vs_format_on_all_noise_ops_diff = test_vs_format_on_all_noise_ops.copy(
    deep=True)
for i in test_vs_format_on_all_noise_ops_diff.index:
    for j in test_vs_format_on_all_noise_ops_diff.columns:
        ix = (i[0], "OriginalData")
        if i[1] != "OriginalData":
            subtract_from = test_vs_format_on_all_noise_ops.loc[ix, j]
            difference = subtract_from - \
                test_vs_format_on_all_noise_ops_diff.loc[i, j]
            if "-" in str(difference):
                str_diff = "{:.2f}".format(difference)
            else:
                str_diff = "+"+"{:.2f}".format(difference)

            test_vs_format_on_all_noise_ops_diff.loc[i, j] = str_diff

In [ ]:
test_vs_format_on_all_noise_ops_diff